# Prior Monte Carlo

In it's simplest form, Monte Carlo boils down to: 1) "draw" many random samples of parameters from the prior probability distribution, 2) run the model, 3) look at the results.

So how do we "draw", or sample, parameters? (Think "draw" as in "drawing a card from a deck"). We need to randomly sample parameter values from a range. This range is defined by the _prior parameter probability distribution_. As we did for FOSM, let's assume that the bounds in the parameter data section define the range of a Gaussian (or normal) distribution, and that the initial values define the mean. 

We will also use the results of the prior MC to choose the truth for our dual experiment where we see if the modeling can bracket the simulated results of an unseen realization...

<font color="#ff7700ff">
En su forma más simple, <code>Monte Carlo</code> es: 1) "sacar" muchas muestras aleatorias de parámetros de la distribución de probabilidad previa, 2) correr el modelo, 3) mirar los resultados.

¿Cómo "sacamos" o muestreamos parámetros? (Piensa en "sacar" como sacar una carta de una mazo). Hay que muestrear valores de parámetros de manera aleatoria dentro de un rango. Ese rango lo define la <code>distribución de probabilidad previa</code> de los parámetros.

También vamos a usar los resultados del MC previo para elegir la "verdad" en nuestro experimento dual, donde vemos si el modelo puede acotar los resultados simulados de una realización que no ha visto...
</font>

In [ ]:


import os
import shutil
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt;

import sys
import pyemu
import flopy


# specify the temporary working folder
t_d = os.path.join('pst_template_pmc')
# get the previously generated PEST dataset
org_t_d = os.path.join('pst_template')
if not os.path.exists(org_t_d):
    raise Exception()
if os.path.exists(t_d):
    shutil.rmtree(t_d)
shutil.copytree(org_t_d,t_d)

In [ ]:
pst = pyemu.Pst(os.path.join(t_d,"pest.pst"))

In [ ]:
pst.observation_data["weight"] = 0.0

In [ ]:
pst.pestpp_options["ies_num_reals"] = 200
pst.pestpp_options["save_binary"] = True
pst.control_data.noptmax = -1

In [ ]:
pst.write(os.path.join(t_d,"pest.pst"),version=2)

In [ ]:
num_workers=10
m_d = "master_prior_mc"

In [ ]:
pyemu.os_utils.start_workers(t_d, # the folder which contains the "template" PEST dataset
                            'pestpp-ies', #the PEST software version we want to run
                            'pest.pst', # the control file to use with PEST
                            num_workers=num_workers, #how many agents to deploy
                            worker_root='.', #where to deploy the agent directories; relative to where python is running
                            master_dir=m_d, #the manager directory
                            )

# Exploring prior forecast uncertainty

At this point, we are prepared to check the uncertainty of the predictions before "calibration".  

In [ ]:
pst = pyemu.Pst(os.path.join(m_d,"pest.pst"))

In [ ]:
oe = pst.ies.obsen

In [ ]:
forecasts = [f.strip() for f in pst.pestpp_options["forecasts"].split(',')]
forecasts.sort()
forecasts

In [ ]:
for f in forecasts:
    ax = oe.loc[:,f].plot(kind="hist",fc="0.5",alpha=0.5,density=True)
    ax.set_title(f)
    ax.set_yticks([])
    ax.set_ylabel("")
    plt.show()

there you go! you just did uncertainty analysis!

## Let's define the truth

In this synthetic case we are going to use one of the realizations as our "truth" and use it to pull some big and beautiful "measurement" observations for data assimilation.

<font color="#ff7700ff">
En este caso sintético vamos a usar una de las realizaciones como nuestra <code>verdad</code> y la vamos a usar para sacar un set de datos de observación para asimilación de datos.
</font>

In [ ]:
pst = pyemu.Pst(os.path.join(m_d,"pest.pst"))

In [ ]:
oe = pst.ies.obsen

In [ ]:
forecasts = [f.strip() for f in pst.pestpp_options["forecasts"].split(',')]
forecasts.sort()
forecasts

In [ ]:
#oe = oe.loc[oe[forecasts[-1]]<80,:]

In [ ]:

vals = oe.loc[:,forecasts[0]].copy()
vals.sort_values(inplace=True)
vals

In [ ]:
quant = np.quantile(vals.values,0.95)
quant

In [ ]:
diff = np.abs(vals - quant)
truth_real = diff.idxmin()
truth_real,vals.loc[truth_real]

In [ ]:
oe.loc[truth_real,:]

In [ ]:
for f in forecasts:
    ax = oe.loc[:,f].plot(kind="hist",fc="0.5",alpha=0.5,density=True)
    ylim = ax.get_ylim()
    v = oe.loc[truth_real,f]
    ax.plot([v,v],ylim,"r--",lw=2)
    ax.set_title(f)
    ax.set_yticks([])
    ax.set_ylabel("")
    plt.show()

We have the observation values for the truth. Let's run that truth thru the  model so we can visual what it looks like

Let's do that and run a test with noptmax zero.

<font color="#ff7700ff">
Ya tenemos los valores de observación para la <code>verdad</code>. 

Hagamos eso y corramos un test con <code>noptmax</code> en cero.
</font>

In [ ]:
pe = pst.ies.paren0
truth_pvals = pe.loc[truth_real,:].values.flatten()
truth_pvals.shape,pst.npar

In [ ]:
pe.index

In [ ]:
truth_pvals

In [ ]:
# make a copy of the original values
org_pvals = pst.parameter_data.loc[pe.columns,"parval1"].copy()
# replace parval1 with our truth
pst.parameter_data.loc[pe.columns,"parval1"] = truth_pvals
pst.control_data.noptmax = 0
pst.write(os.path.join(m_d,"truth.pst"),version=2)

In [ ]:
pyemu.os_utils.run("pestpp-ies truth.pst",cwd=m_d)

Lets checkout the truth HK array:

In [ ]:
df = pd.read_csv(os.path.join(m_d,"mult2model_info.csv"))
kh1_df = df.loc[df.model_file.str.contains("npf_k"),:]
kh1_df
org_arr = np.loadtxt(os.path.join(m_d,kh1_df.org_file.iloc[0]))
inp_arr = np.loadtxt(os.path.join(m_d,kh1_df.model_file.iloc[0]))
mlt_arrs = [np.loadtxt(os.path.join(m_d,afile)) for afile in kh1_df.mlt_file if not pd.isna(afile)]
arrs = [org_arr]
arrs.extend(mlt_arrs)
arrs.append(inp_arr)

mlt_arrs
names = ["org"]
names.extend([mf.split('.')[0].split('_')[-1] for mf in kh1_df.mlt_file if not pd.isna(mf)])
names.append("MF6 input")
fig,axes = plt.subplots(1,len(arrs),figsize=(5*len(arrs),5))
for i,ax in enumerate(axes.flatten()):
    arr = np.log10(arrs[i])
    cb = ax.imshow(arr)
    plt.colorbar(cb,ax=ax, fraction=0.046, pad=0.04)
    ax.set_title(names[i],loc="left")
plt.tight_layout()    

In [ ]:
pst.set_res(os.path.join(m_d,"truth.base.rei"))

## Let's do some initial weighting

Here we’ll show how to do some simple weighting with PyEMU. These weights are just for visibility in the objective function and differ to the ones that represent observation noise, we’ll get to those in the next tutorial.

<font color="#ff7700ff">
Aquí vamos a mostrar cómo hacer un <code>weighting</code> simple con <code>PyEMU</code>. Estos pesos son solo para que se vean en la función objetivo y son distintos a los que representan el ruido de observación, esos los vemos en el próximo tutorial.
</font>

In [ ]:
par = pst.parameter_data
wpar = par.loc[(par.parnme.str.contains("mar")) | (par.parnme.str.contains("dewater")),:].copy()
wpar["i"] = wpar.idx1.astype(int)
wpar["j"] = wpar.idx2.astype(int)
ijs = set([(i,j) for i,j in zip(wpar.i,wpar.j)])

obs = pst.observation_data
obs["weight"] = 0.0

obs.loc[:,"obsval"] = pst.res.loc[pst.obs_names,"modelled"].values
hobs = obs.loc[obs.obsnme.str.contains("hdslay1_t1"),:].copy()
assert hobs.shape[0] > 0
hobs['ij'] = hobs.apply(lambda x: (int(x.i),int(x.j)),axis=1)
nzhobs = hobs.loc[hobs.ij.isin(ijs)]
assert len(nzhobs) == len(ijs)
obs.loc[nzhobs.obsnme,"weight"] = 10.0 #implies sigma = 0.1
drnobs = obs.loc[(obs.usecol=="drn-gde") & (obs.time=="1"),:]
assert len(drnobs) == 1
obs.loc[drnobs.obsnme[0],"weight"] = 1./(np.abs(obs.loc[drnobs.obsnme[0],"obsval"]) * 0.2)
assert pst.nnz_obs == len(ijs) + 1

In [ ]:
pst.parameter_data["parval1"] = org_pvals
pst.write(os.path.join(t_d,"pest.pst"),version=2)

We also should remove the truth from the prior ensemble to make sure we arent cheating ;)

<font color="#ff7700ff">
También hay que sacar la <code>verdad</code> del ensemble previo para no hacer trampa 😉. Si la dejamos, el modelo podría ver la respuesta. Hay que mantener el ensemble solo con realizaciones plausibles, pero que no incluyan la verdad.
</font>



In [ ]:
pe.drop(truth_real,inplace=True)
if "base" in pe.index:
    pe.drop("base",inplace=True)

In [ ]:
pst.pestpp_options

In [ ]:
pyemu.ParameterEnsemble(df=pe,pst=pst).to_binary(os.path.join(t_d,"prior_pe.jcb"))